In [ ]:
import tensorflow as tf

# 0 、文件位置

In [ ]:
trainset_path = "./image/train"
testset_path = "./image/test"
output_path = "./output"

# 一、创建 TFRecords文件

In [ ]:
import glob
import os
from itertools import groupby
from scipy import misc

In [ ]:
# class 信息
breeds_with_path = glob.glob(trainset_path + "/*")
breeds = list(map(lambda x:x.split("/")[3],breeds_with_path))
breeds_dict = dict(zip(breeds, [x for x in range(len(breeds))]))

In [ ]:
print(breeds_dict["cat"])
print(breeds_dict["bird"])
print(breeds_dict["dog"])

In [ ]:
def create_record(dataset_path,output_location):
    # 获取数据集下所有文件路径
    dataset_files = glob.glob(dataset_path + "/*/*.jpg")
    # 分离出claess信息,得到（class,file_path）对的列表
    image_filepath_with_breed = list(map(lambda file_path:(file_path.split("/")[3],file_path),dataset_files))
    
    #每个TFRecord文件保存10个图片
    nums_per_file = 10
    writer = None
    
    # 记录当前读到的图像index
    current_index = 0
    for breed, file_path in image_filepath_with_breed:
        if current_index % nums_per_file == 0:
            if writer:
                print("current_index = ",current_index,"\n")
                writer.close()
            
            # 格式化字符串
            record_filename = "{output_location}{current_index}.tfrecords".format(
            output_location = output_location,
            current_index = current_index)
            
            writer = tf.python_io.TFRecordWriter(record_filename)
            
        current_index += 1
        
        # 读取图像并转换为tf.example格式
        image = misc.imread(file_path)
            
        # 图像转换为Byte型
        image_raw = image.tobytes()
        
        example = tf.train.Example(features = 
                                  tf.train.Features(feature = {
                    "label": tf.train.Feature(int64_list = tf.train.Int64List(value = [breeds_dict[breed]])),
                    "image_raw": tf.train.Feature(bytes_list = tf.train.BytesList(value = [image_raw]))   
                }     
            )    
        )
        
        # 写入文件
        writer.write(example.SerializeToString())
        
    # 关闭最后一个文件
    writer.close()

In [ ]:
create_record(trainset_path,output_path + "/training-images/")
create_record(testset_path,output_path + "/testing-images/")

# 二、 读取TFRecord 文件